<a href="https://colab.research.google.com/github/z-guard/analysis/blob/main/notebooks/town_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 町の特徴量


In [1]:
!pip install -q japanize-matplotlib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
DIR_NAME = '/content/drive/MyDrive/z-gard/data'

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [5]:
def rank_score(sr_data):
    rank = sr_data.rank()
    rmax = rank.max()
    rmin = rank.min()
    max_val = 5
    x = (rank - rmin)/(rmax - rmin) * max_val
    return np.clip(x, 0, max_val)

### 人口マスタ

In [6]:
df_population = pd.read_csv(os.path.join(DIR_NAME, 'population_master.csv')).set_index('town_id')
print(df_population.shape)

(3145, 59)


In [7]:
df_population['人口スコア'] = rank_score(df_population['人口総数（人）']).round(2)
df_population['人口密度スコア'] = rank_score(df_population['人口密度(人/km2)']).round(2)
df_population['世帯総数スコア'] = rank_score(df_population['世帯総数']).round(2)

In [8]:
df_population.columns

Index(['市区町村名', '町名', '町丁目', '人口総数（人）', '面積（km2）', '人口密度(人/km2)', '0〜4歳（人）',
       '5〜9歳（人）', '10〜14歳（人）', '15〜19歳（人）', '20〜24歳（人）', '25〜29歳（人）',
       '30〜34歳（人）', '35〜39歳（人）', '40〜44歳（人）', '45〜49歳（人）', '50〜54歳（人）',
       '55〜59歳（人）', '60〜64歳（人）', '65〜69歳（人）', '70〜74歳（人）', '75〜79歳（人）',
       '80〜84歳（人）', '85〜89歳（人）', '90〜94歳（人）', '95〜99歳（人）', '100歳以上（人）',
       '年齢不詳（人）', '平均年齢（歳）', '外国人（人）', '対象', '世帯総数', '1人世帯', '2人世帯', '3人世帯',
       '4人世帯', '5人以上世帯', '世帯人員（人）', '1世帯当たり人員（人）', '労働力人口割合%', '非労働力人口割合%',
       '非労働力_専業主婦割合%', '非労働力_学生割合%', '非労働力_幼児・高齢者割合%', '労働力_サラリーマン割合%',
       '労働力_役員割合%', '労働力_自営業者割合%', '労働力_パートタイム割合%', '労働力_学生アルバイト割合%',
       '持ち家_割合%', '公営借家_割合%', '民営借家_割合%', '社宅_割合%', '一戸建_割合%', '長屋建_割合%',
       '共同住宅_１・２階建_割合%', '共同住宅_３～５階建_割合%', '共同住宅_６～10階建_割合%',
       '共同住宅_11階建以上_割合%', '人口スコア', '人口密度スコア', '世帯総数スコア'],
      dtype='object')

### 人口分布

In [9]:
df_population_score = pd.read_csv(os.path.join(DIR_NAME, 'population_score.csv')).set_index('town_id')
print(df_population_score.shape)

(3145, 31)


In [10]:
df_population_score.columns

Index(['乳幼児割合スコア', '小学生割合スコア', '中学生割合スコア', '高校生割合スコア', '10代割合スコア', '20代割合スコア',
       '30代割合スコア', '40代割合スコア', '50代割合スコア', '60代割合スコア', '10代_子割合スコア',
       '20代_子割合スコア', '30代_子割合スコア', '40代_子割合スコア', '50代_子割合スコア', '戸建割合スコア',
       'マンション割合スコア', '低層マンション割合スコア', '中層マンション割合スコア', '高層マンション割合スコア',
       '持ち家割合スコア', '賃貸割合スコア', '団地割合スコア', '単身割合スコア', 'カップル割合スコア', 'ファミリー割合スコア',
       '大家族割合スコア', '学生指数スコア', '専業主婦指数スコア', '上流指数スコア', '中流指数スコア'],
      dtype='object')

### 利便性

In [11]:
df_convenience_score = pd.read_csv(os.path.join(DIR_NAME, 'convenience_score.csv')).set_index('town_id')
print(df_convenience_score.shape)

(3145, 23)


In [12]:
df_convenience_score['チェーン店(数)'] = df_convenience_score['カフェ(チェーン店)(数)'] + df_convenience_score['レストラン(チェーン店)(数)']

In [13]:
feature_list = [col for col in df_convenience_score.columns if col.endswith('(数)')]
for feature in feature_list:
    df_convenience_score[f'{feature}スコア'] = rank_score(df_convenience_score[feature]).round(2)

df_convenience_score['最寄駅(距離)スコア'] = rank_score((-1) * df_convenience_score['最寄駅(距離)']).round(2)
df_convenience_score['平均所要(時間)スコア'] = rank_score((-1) * df_convenience_score['平均所要(時間)']).round(2)

In [14]:
df_convenience_score.columns

Index(['コンビニ(数)', 'スーパー(数)', '商店街(数)', '銭湯(数)', 'カラオケ(数)', 'ネットカフェ(数)',
       'クリーニング(数)', '本DVD(数)', '衣料品(数)', 'ビューティーサロン(数)', '100円ショップ(数)',
       'カフェ(数)', 'カフェ(チェーン店)(数)', 'ファストフード(数)', 'レストラン(数)', 'レストラン(チェーン店)(数)',
       '飲み屋(数)', '最寄駅(距離)', '平均所要(時間)', '買い物スコア', '駅スコア', '飲食店スコア', '利便性スコア',
       'チェーン店(数)', 'コンビニ(数)スコア', 'スーパー(数)スコア', '商店街(数)スコア', '銭湯(数)スコア',
       'カラオケ(数)スコア', 'ネットカフェ(数)スコア', 'クリーニング(数)スコア', '本DVD(数)スコア', '衣料品(数)スコア',
       'ビューティーサロン(数)スコア', '100円ショップ(数)スコア', 'カフェ(数)スコア', 'カフェ(チェーン店)(数)スコア',
       'ファストフード(数)スコア', 'レストラン(数)スコア', 'レストラン(チェーン店)(数)スコア', '飲み屋(数)スコア',
       'チェーン店(数)スコア', '最寄駅(距離)スコア', '平均所要(時間)スコア'],
      dtype='object')

### 周辺環境

In [15]:
df_environment_score = pd.read_csv(os.path.join(DIR_NAME, 'environment_score.csv')).set_index('town_id')
print(df_environment_score.shape)

(3145, 6)


In [16]:
df_environment_score.columns

Index(['公園スコア', '運動場スコア', '体育館スコア', 'プールスコア', 'フィットネススコア', '周辺環境スコア'], dtype='object')

### 地価スコア

In [17]:
df_land_price_score = pd.read_csv(os.path.join(DIR_NAME, 'land_price_score.csv')).rename(columns={'地域ID': 'town_id'}).set_index('town_id')
print(df_land_price_score.shape)

(3145, 6)


In [18]:
df_land_price_score.columns

Index(['地価', '地価スコア', '家賃予測_ワンルーム', '家賃予測_1K', '家賃予測_1LDK', '家賃予測_2LDK'], dtype='object')

### 名所旧跡

In [19]:
df_famous_place_score = pd.read_csv(os.path.join(DIR_NAME, 'famous_place_score.csv')).set_index('town_id')
print(df_famous_place_score.shape)

(3145, 6)


In [20]:
feature_list = [col for col in df_famous_place_score.columns if col.endswith('(数)')]
for feature in feature_list:
    df_famous_place_score[f'{feature}スコア'] = rank_score(df_famous_place_score[feature]).round(2)

In [21]:
df_famous_place_score.columns

Index(['寺(数)', '神社(数)', '旧跡(数)', '並木道(数)', '橋(数)', '墓地(数)', '寺(数)スコア',
       '神社(数)スコア', '旧跡(数)スコア', '並木道(数)スコア', '橋(数)スコア', '墓地(数)スコア'],
      dtype='object')

## 川沿い

In [22]:
df_riverside_score = pd.read_csv(os.path.join(DIR_NAME, 'river_side_score.csv')).set_index('town_id')
print(df_riverside_score.shape)

(3145, 2)


In [23]:
df_riverside_score['川(数)スコア'] = rank_score(df_riverside_score['川(数)']).round(2)
df_riverside_score['川(距離)スコア'] = rank_score((-1) * df_riverside_score['川(距離)']).round(2)

In [24]:
df_riverside_score.columns

Index(['川(数)', '川(距離)', '川(数)スコア', '川(距離)スコア'], dtype='object')

### 学校

In [25]:
df_school_score = pd.read_csv(os.path.join(DIR_NAME, 'public_school_score.csv')).set_index('town_id')
print(df_school_score.shape)

(3145, 11)


In [26]:
df_school_score.columns

Index(['小学校(最近傍距離)', '名門小学校', '中学校(最近傍距離)', '幼稚園(最近傍距離)', '学習塾(数)', '大学(数)',
       '小学校スコア', '中学校スコア', '幼稚園スコア', '学習塾スコア', '大学スコア'],
      dtype='object')

### 子育て

In [27]:
df_childcare_score = pd.read_csv(os.path.join(DIR_NAME, 'childcare_score.csv')).set_index('town_id')
print(df_childcare_score.shape)

(3145, 4)


In [28]:
df_childcare_score.columns

Index(['幼稚園スコア', '保育園スコア', '児童館スコア', '子育てスコア'], dtype='object')

### 全てマージ

In [29]:
df_town_feature = pd.concat([
    df_population,
    df_population_score,
    df_convenience_score,
    df_environment_score,
    df_land_price_score,
    df_famous_place_score,
    df_riverside_score,
    df_school_score,
    df_childcare_score
], axis=1)
print(df_town_feature.shape)
assert len(df_town_feature) == len(df_population)

(3145, 180)


In [30]:
basic_cols = ['市区町村名', '町名', '町丁目', '対象']
score_list = [col for col in df_town_feature.columns if col.endswith('スコア')]
df_town_score = df_town_feature[basic_cols + score_list].copy()

### 各種スコア
- 閑静、繁華街、郊外、団地
- 戸建、低層マンション、高層マンション、アパート
- おしゃれ、歴史・文化的、下町、セレブ
- 自然、川、駅近、商店街、散歩道
- 学生、単身、カップル、子育てファミリー
- 賃貸、購入

In [31]:
def get_score(df_data, score_name, score_recipe, household=0):
    df_data[score_name] = 0
    for key, val in score_recipe.items():
        df_data[score_name] += df_data[key] * val
    # 世帯が少ないところは足切り
    df_data.loc[df_data['世帯総数スコア'] < household, score_name] = 0
    df_data[score_name] = rank_score(df_data[score_name]).round(2)

In [32]:
output_score_list = []

In [33]:
score_name = '閑静スコア'
score_recipe = {
    '世帯総数スコア': 0,
    '人口密度スコア': -0.3,
    '戸建割合スコア': 1,
    '低層マンション割合スコア': 0.7,
    '持ち家割合スコア': 0.7,
    '団地割合スコア': -0.7,
    'ファミリー割合スコア': 0.3,
    '単身割合スコア': -0.3,
    '学生指数スコア': -0.3,
    'チェーン店(数)スコア': -0.5,
    '並木道(数)スコア': 0.5,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 0.1)

In [34]:
score_name = 'セレブスコア'
score_recipe = {
    '世帯総数スコア': 0,
    '人口密度スコア': 0,
    '専業主婦指数スコア': 1,
    '上流指数スコア': 1,
    'ビューティーサロン(数)スコア': 0.3,
    'チェーン店(数)スコア': -0.5,
    '地価スコア': 1,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 0.1)

In [35]:
score_name = '繁華街スコア'
score_recipe = {
    '世帯総数スコア': 0,
    '人口密度スコア': 0.5,
    '戸建割合スコア': -0.5,
    '中層マンション割合スコア': 0.7,
    '高層マンション割合スコア': 0.5,
    '賃貸割合スコア': 0.5,
    '単身割合スコア': 0.3,
    '商店街(数)スコア': 0.5,
    'コンビニ(数)スコア': 0.7,
    '飲み屋(数)スコア': 1,
    'チェーン店(数)スコア': 0.5,
    '地価スコア': 0.7,
    '最寄駅(距離)スコア': 0.7,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [36]:
score_name = '団地スコア'
score_recipe = {
    '世帯総数スコア': 0,
    '世帯総数スコア': 0.5,
    '人口密度スコア': -0.3,
    '戸建割合スコア': -0.7,
    '低層マンション割合スコア': 0.5,
    '中層マンション割合スコア': 0.7,
    '高層マンション割合スコア': -0.5,
    '賃貸割合スコア': 0.5,
    '団地割合スコア': 1,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [37]:
score_name = '戸建スコア'
score_recipe = {
    '世帯総数スコア': 0,
    '人口密度スコア': -0.1,
    '戸建割合スコア': 1,
    '中層マンション割合スコア': -0.5,
    '高層マンション割合スコア': -0.5,
    'ファミリー割合スコア': 0.3,
    '持ち家割合スコア': 0.5,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [38]:
score_name = '低層マンションスコア'
score_recipe = {
    '世帯総数スコア': 0,
    '人口密度スコア': 0.3,
    '戸建割合スコア': -0.7,
    '低層マンション割合スコア': 1,
    '中層マンション割合スコア': 0.7,
    '高層マンション割合スコア': -0.5,
    'ファミリー割合スコア': 0.3,
    '持ち家割合スコア': 0.3,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [39]:
score_name = '高層マンションスコア'
score_recipe = {
    '世帯総数スコア': 0,
    '人口密度スコア': 0.3,
    '戸建割合スコア': -0.7,
    '低層マンション割合スコア': -0.5,
    '中層マンション割合スコア': 0.5,
    '高層マンション割合スコア': 1,
    'ファミリー割合スコア': 0.5,
    '上流指数スコア': 0.5,
    '地価スコア': 1,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 0)

In [40]:
score_name = 'アパートスコア'
score_recipe = {
    '世帯総数スコア': 0,
    '人口密度スコア': 0.3,
    '戸建割合スコア': -0.7,
    '低層マンション割合スコア': 1,
    '中層マンション割合スコア': -0.7,
    '高層マンション割合スコア': -1,
    '賃貸割合スコア': 0.7,
    '団地割合スコア': 0.7,
    '上流指数スコア': -0.7,
    '中流指数スコア': -0.5,
    '単身割合スコア': 0.5,
    '学生指数スコア': 0.5,
    '地価スコア': -0.3,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [41]:
score_name = 'おしゃれスコア'
score_recipe = {
    '世帯総数スコア': 0,
    '人口密度スコア': 0.3,
    '20代割合スコア': 0.7,
    '30代割合スコア': 0.7,
    '低層マンション割合スコア': 0.5,
    '中層マンション割合スコア': 0.7,
    'カップル割合スコア': 0.3,
    '上流指数スコア': 0.7,
    '中流指数スコア': 0.5,
    '地価スコア': 0.7,
    'ビューティーサロン(数)スコア': 0.5,
    'カフェ(数)スコア': 0.5,
    'カフェ(チェーン店)(数)スコア': 0.5,
    '並木道(数)スコア': 0.5,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [42]:
score_name = '歴史文化スコア'
score_recipe = {
    '世帯総数スコア': 0.5,
    '人口密度スコア': 0.3,
    '地価スコア': 0.5,
    '寺(数)スコア': 0.5,
    '神社(数)スコア': 0.5,
    '旧跡(数)スコア': 0.7,
    '並木道(数)スコア': 0.5,
    '大学スコア': 0.3,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [43]:
score_name = '下町スコア'
score_recipe = {
    '世帯総数スコア': 0.5,
    '人口密度スコア': 0.7,
    '50代割合スコア': 0.3,
    '60代割合スコア': 0.3,
    '上流指数スコア': -1,
    '中流指数スコア': -0.7,
    '地価スコア': -0.5,
    '寺(数)スコア': 0.5,
    '神社(数)スコア': 0.5,
    '旧跡(数)スコア': 0.5,
    '並木道(数)スコア': 0.3,
    '商店街(数)スコア': 0.7,
    '銭湯(数)スコア': 0.5,
    '飲み屋(数)スコア': 0.5,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [44]:
score_name = '都会スコア'
score_recipe = {
    '世帯総数スコア': 0.3,
    '人口密度スコア': 0.5,
    '上流指数スコア': 0.5,
    '飲食店スコア': 0.5,
    'コンビニ(数)スコア': 0.7,
    '飲み屋(数)スコア': 0.5,
    'チェーン店(数)スコア': 0.7,
    '最寄駅(距離)スコア': 0.3,
    '平均所要(時間)スコア': 1,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [45]:
score_name = '郊外スコア'
score_recipe = {
    '世帯総数スコア': 0.5,
    '人口密度スコア': 0,
    'ファミリー割合スコア': 0.5,
    '平均所要(時間)スコア': -1,
    '公園スコア': 0.7,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [46]:
score_name = '川スコア'
score_recipe = {
    '世帯総数スコア': 0.7,
    '人口密度スコア': 0.5,
    '川(数)スコア': 1,
    '川(距離)スコア': 0.7,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 2)

In [47]:
score_name = '自然スコア'
score_recipe = {
    '世帯総数スコア': 0.5,
    '人口密度スコア': 0,
    '川(距離)スコア': 0.3,
    '公園スコア': 0.7,
    '平均所要(時間)スコア': -1,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 2)

In [48]:
score_name = '駅近スコア'
score_recipe = {
    '世帯総数スコア': 0.7,
    '人口密度スコア': 0.5,
    '駅スコア': 1,
    '飲食店スコア': 0.7,
    '地価スコア': 0.7,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 2)

In [49]:
score_name = '商店街スコア'
score_recipe = {
    '世帯総数スコア': 0.7,
    '人口密度スコア': 0.5,
    '商店街(数)スコア': 1,
    '飲食店スコア': 0.7,
    '100円ショップ(数)スコア': 0.3,
    'ファストフード(数)スコア': 0.3,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [50]:
score_name = '散歩道スコア'
score_recipe = {
    '世帯総数スコア': 0.7,
    '人口密度スコア': 0.5,
    '上流指数スコア': 0.5,
    '中流指数スコア': 0.3,
    '地価スコア': 0.5,
    '並木道(数)スコア': 1,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [51]:
score_name = '学生スコア'
score_recipe = {
    '世帯総数スコア': 0.3,
    '人口密度スコア': 0,
    '20代割合スコア': 0.5,
    '単身割合スコア': 0.7,
    'ファミリー割合スコア': -0.5,
    '学生指数スコア': 1,
    '戸建割合スコア': -0.7,
    '賃貸割合スコア': 0.7,
    '飲食店スコア': 0.5,
    'コンビニ(数)スコア': 0.7,
    'スーパー(数)スコア': 0.5,
    '地価スコア': -0.5,
    '大学スコア': 0.3,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [52]:
score_name = '単身スコア'
score_recipe = {
    '世帯総数スコア': 0.3,
    '人口密度スコア': 0,
    '20代割合スコア': 0.5,
    '30代割合スコア': 0.3,
    '単身割合スコア': 1,
    'ファミリー割合スコア': -0.5,
    '戸建割合スコア': -0.7,
    '賃貸割合スコア': 0.7,
    '飲食店スコア': 0.5,
    'コンビニ(数)スコア': 0.7,
    'スーパー(数)スコア': 0.5,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [53]:
score_name = 'カップルスコア'
score_recipe = {
    '世帯総数スコア': 0.3,
    '人口密度スコア': 0,
    '20代割合スコア': 0.3,
    '30代割合スコア': 0.3,
    '単身割合スコア': -0.7,
    'カップル割合スコア': 1,
    'ファミリー割合スコア': -0.5,
    '賃貸割合スコア': 0.7,
    '飲食店スコア': 0.5,
    'コンビニ(数)スコア': 0.7,
    'スーパー(数)スコア': 0.5,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [54]:
score_name = '子育てファミリースコア'
score_recipe = {
    '世帯総数スコア': 0.5,
    '人口密度スコア': 0,
    '20代_子割合スコア': 0.3,
    '30代_子割合スコア': 0.3,
    '単身割合スコア': -0.7,
    'ファミリー割合スコア': 1,
    '学生指数スコア': -0.7,
    'スーパー(数)スコア': 0.5,
    '子育てスコア': 0.7,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [55]:
score_name = '賃貸スコア'
score_recipe = {
    '世帯総数スコア': 0.5,
    '人口密度スコア': 0,
    '持ち家割合スコア': -0.3,
    '賃貸割合スコア': 1,
    '団地割合スコア': 0.7,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [56]:
score_name = '購入スコア'
score_recipe = {
    '世帯総数スコア': 0.5,
    '人口密度スコア': 0,
    '持ち家割合スコア': 1,
    '賃貸割合スコア': -0.5,
    '団地割合スコア': -0.7,
}
output_score_list.append(score_name)
get_score(df_town_score, score_name, score_recipe, 1)

In [57]:
# df_town_score[basic_cols + [score_name] + list(score_recipe.keys())].sort_values(score_name, ascending=False).head(30)

### 保存

In [58]:
df_town_score[output_score_list].to_csv(os.path.join(DIR_NAME, 'town_feature.csv'), index=True, encoding='utf_8_sig')